In [ ]:
# Import required packages
%pip install langchain langchain-community pypdf langchain_google_genai faiss-cpu

In [ ]:
import os
import glob
import signal
import sys
import random
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain.schema import HumanMessage
from langchain.chains import RetrievalQA
from langchain.prompts import ChatPromptTemplate

In [8]:
# Load all PDFs from the specified folder
pdf_folder_path = "../Data/"
all_pdf_paths = glob.glob(os.path.join(pdf_folder_path, "*.pdf"))

# Load each PDF document and split text
documents = []
for pdf_path in all_pdf_paths:
    loader = PyPDFLoader(pdf_path)
    pdf_docs = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    documents.extend(text_splitter.split_documents(pdf_docs))

print(f"Total loaded document chunks: {len(documents)}")

Total loaded document chunks: 1518


In [10]:
random_chunk = random.choice(documents)

# Menampilkan chunk yang dipilih
print("Randomly selected document chunk:")
print(random_chunk)

Randomly selected document chunk:
page_content='- 35 - 
             
       
 d. Assist  (bimbing)  
Bantu pasien dengan rencana berhenti merokok, menyediakan 
konseling praktis, merekomendasikan penggunaan farmakoterapi. 
e. Arrange (atur)  
Buat jadwal kontak lebih lanjut. 
3. Obat-obatan a. Bronkodilator : Diberikan secara tunggal atau kombinasi dari ketiga jenis bronkodilator dan disesuaikan dengan klasifikasi derajat berat penyakit. Pemilihan bentuk obat diutamakan dalam bentuk 
inhalasi. Saat pemberian terapi inhalasi sebaiknya tidak' metadata={'source': '../Data\\COPD-Kemenkes.pdf', 'page': 34}


In [19]:
from dotenv import load_dotenv
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

In [ ]:
# Set up embeddings and LLM with Google Gemini API
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=GEMINI_API_KEY)
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", google_api_key=GEMINI_API_KEY)

In [12]:
# Create FAISS vector database from documents
vector_db = FAISS.from_documents(documents, embeddings)
retriever = vector_db.as_retriever(search_type="similarity", search_kwargs={"k": 5})

In [13]:
# Define RAG prompt templates for different recommendations
def generate_treatment_prompt(query, context, recommendation_type):
    prompt = f"""
    Anda adalah seorang ahli kesehatan khususnya penyakit PPOK (penyakit paru obstruktif kronik) yang membantu petugas kesehatan untuk memberikan rekomendasi pengobatan kepada pasien berdasarkan informasi yang tersedia.

    **Profil dan Riwayat Pasien**:
    {query}

    **Riwayat Medis dan Keterangan Medis**:
    {context}

    Berdasarkan informasi di atas, berikan rekomendasi pengobatan yang singkat namun spesifik dan jelas yang meliputi:
    1. Obat yang disarankan beserta dosisnya (jika mungkin).
    2. Metode pengobatan yang sesuai.
    3. Langkah perawatan yang harus dilakukan oleh petugas medis terhadap pasien.
    """
    return prompt

In [14]:
def generate_lifestyle_prompt(query, context, recommendation_type):
    prompt = f"""
    Anda adalah seorang ahli kesehatan khususnya penyakit PPOK (penyakit paru obstruktif kronik) yang membantu petugas kesehatan memberikan rekomendasi pola hidup sehat kepada pasien yang terindikasi TB atau tidak dengan informasi berikut.

    **Profil dan Riwayat Pasien**:
    {query}

    **Riwayat Medis dan Keterangan Medis**:
    {context}

    Berdasarkan informasi di atas, berikan rekomendasi pola hidup yang singkat namun spesifik yang mencakup:
    1. Aktivitas fisik yang aman dan direkomendasikan (misalnya, jenis olahraga dan frekuensinya).
    2. Pola makan dan jenis makanan yang sebaiknya dikonsumsi dan dihindari (contoh: makanan yang meningkatkan imunitas).
    3. Kebiasaan sehari-hari yang dapat membantu pemulihan, termasuk tips manajemen stres dan tidur.
    4. Instruksi khusus untuk menjaga kebersihan dan mencegah penularan.
    """
    return prompt

In [15]:
def generate_followup_prompt(query, context, recommendation_type):
    prompt = f"""
    Anda adalah seorang ahli kesehatan khususnya penyakit PPOK (penyakit paru obstruktif kronik) yang memberikan rekomendasi penanganan lanjutan bagi petugas kesehatan untuk pasien yang terindikasi TB atau tidak dengan informasi berikut.

    **Profil dan Riwayat Pasien**:
    {query}

    **Riwayat Medis dan Keterangan Medis**:
    {context}

    Berdasarkan informasi di atas, berikan rekomendasi penanganan lanjutan yang singkat namun spesifik mencakup:
    1. Jadwal kontrol kesehatan atau pemeriksaan lanjutan yang disarankan.
    2. Pengujian tambahan atau pemeriksaan yang mungkin diperlukan (contoh: X-ray atau tes laboratorium).
    3. Tanda atau gejala yang perlu diwaspadai sebagai indikasi komplikasi.
    4. Saran untuk pemulihan yang berkelanjutan, seperti adaptasi pola hidup, manajemen stres, dan dukungan sosial yang dibutuhkan.
    """
    return prompt

In [16]:
# Prompt untuk memilih jenis rekomendasi
def choose_recommendation_type():
    print("Pilih Jenis Rekomendasi:")
    print("1. Rekomendasi Pengobatan")
    print("2. Rekomendasi Pola Hidup")
    print("3. Rekomendasi Penanganan Lanjutan")
    print("4. Keluar")
    choice = input("Masukkan pilihan Anda (1/2/3/4): ")
    return choice

#### Output Rekomendasi RAG

In [17]:
# Main loop
while True:
    profil_pasien = input("Masukkan Profil Pasien (umur, jenis kelamin, dll): ")
    if profil_pasien.lower() == 'exit':
        print("Exiting the program. Goodbye!")
        break

    riwayat_pasien = input("Masukkan Riwayat Pasien: ")
    pola_hidup = input("Masukkan Pola Hidup Pasien: ")
    hasil_pred = input("Masukkan Hasil Prediksi (PPOK/Tidak): ")

    # Gabungkan data pasien sebagai query untuk konteks rekomendasi
    query = f"Profil pasien: {profil_pasien}. Riwayat: {riwayat_pasien}. Pola Hidup:{pola_hidup}. Hasil CT Scan: {hasil_pred}."
    context = "\n".join([result.page_content for result in retriever.get_relevant_documents(query)])

    while True:
        choice = choose_recommendation_type()

        if choice == '1':
            # Use generate_prompt instead of generate_rag_prompt
            prompt = generate_treatment_prompt(query=query, context=context, recommendation_type="Rekomendasi Pengobatan")
        elif choice == '2':
            # Use generate_prompt instead of generate_rag_prompt
            prompt = generate_lifestyle_prompt(query=query, context=context, recommendation_type="Rekomendasi Pola Hidup")
        elif choice == '3':
            # Use generate_prompt instead of generate_rag_prompt
            prompt = generate_followup_prompt(query=query, context=context, recommendation_type="Rekomendasi Penanganan Lanjutan")
        elif choice == '4':
            print("Kembali ke awal. Silakan masukkan informasi pasien baru.")
            break
        else:
            print("Pilihan tidak valid. Coba lagi.")
            continue

        # Buat pesan HumanMessage dan dapatkan hasil dari model LLM
        messages = [HumanMessage(content=prompt)]
        answer = llm(messages=messages)
        answer = answer.content
        print("Jawaban:", answer)

C:\Users\lenovo\AppData\Local\Temp\ipykernel_14040\3363164819.py:14: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  context = "\n".join([result.page_content for result in retriever.get_relevant_documents(query)])


Pilih Jenis Rekomendasi:
1. Rekomendasi Pengobatan
2. Rekomendasi Pola Hidup
3. Rekomendasi Penanganan Lanjutan
4. Keluar


C:\Users\lenovo\AppData\Local\Temp\ipykernel_14040\3363164819.py:37: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  answer = llm(messages=messages)


Jawaban: ## Rekomendasi Pengobatan untuk Pasien PPOK

**Berdasarkan informasi yang tersedia, berikut rekomendasi pengobatan yang singkat namun spesifik dan jelas:**

**1. Obat yang disarankan beserta dosisnya:**

* **Bronkodilator:** 
    * **Inhaler jangka panjang:**  
        * **SABA (Short-Acting Beta Agonist):**  Albuterol 2-4 puffs 4x sehari, sesuai kebutuhan.
        * **LABA (Long-Acting Beta Agonist):**  Salmeterol 50 mcg 2x sehari atau Formoterol 12 mcg 2x sehari.
    * **Anticholinergik:**  
        * **LAMA (Long-Acting Muscarinic Antagonist):** Tiotropium 18 mcg sekali sehari atau Aclidinium 400 mcg 2x sehari.
* **Kortikosteroid:**  
    * **Inhalasi:** Budesonide 400-800 mcg 2x sehari atau Fluticasone 250-500 mcg 2x sehari.
    * **Oral:** Prednisone 40-60 mg/hari selama 5-7 hari (hanya untuk eksaserbasi akut).
* **Obat lain:** 
    * **Antibiotik:** Jika ada infeksi.
    * **Oksigen:** Jika saturasi oksigen rendah.
    * **Rehabilitasi paru:**  Untuk meningkatkan kemampu